In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
train = pd.read_csv("c:/Bigmart/bigmarttrain.csv")
test= pd.read_csv("c:/Bigmart/bigmarttest.csv")
train['source']='train'
test['source']='test'
data= pd.concat([train,test],ignore_index=True)

In [32]:
data.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store,train
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1,train


In [33]:
data.keys()

Index(['Item_Fat_Content', 'Item_Identifier', 'Item_MRP', 'Item_Outlet_Sales',
       'Item_Type', 'Item_Visibility', 'Item_Weight',
       'Outlet_Establishment_Year', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type', 'source'],
      dtype='object')

In [34]:
data.dtypes

Item_Fat_Content              object
Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
Outlet_Location_Type          object
Outlet_Size                   object
Outlet_Type                   object
source                        object
dtype: object

In [35]:
#All the items seem to be of the correct data types
data.shape

(14204, 13)

In [36]:
#checking null values
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [37]:
#Item outlet sales has missing values since those belong to the test set. we ignore that.
# Item weight and oulet size have missing values. outlet size in particular has a lot. Plus is a categorical variable.
#item weight on the other hand is an ratio type. lets see how to impute these.
#looking at unique counts of each column
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
source                           2
dtype: int64

In [38]:
data['Outlet_Size'].value_counts()

Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [39]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [40]:
#dropping columns like product id etc.
#df=data.drop(['Item_Identifier','Outlet_Identifier'], axis=1)

In [41]:
#df.shape

In [42]:
categorical_columns= [x for x in data.dtypes.index if data.dtypes[x]== 'object']
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
for col in categorical_columns:
    print ('\nfrequency of categories for variable %s'%col)
    print (data[col].value_counts())


frequency of categories for variable Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

frequency of categories for variable Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

frequency of categories for variable Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

frequency of categories for variable Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int6

In [44]:
#imputing missing values in Item_weight by avg weight against each item identifier
item_avg_weight= data.pivot_table(values='Item_Weight', index='Item_Identifier')
miss_boolean= data['Item_Weight'].isnull()
data.loc[miss_boolean,'Item_Weight']= data.loc[miss_boolean,'Item_Identifier'].apply(lambda x: item_avg_weight[x])
sum(data['Item_Weight'].isnull())

KeyError: 'FDP10'

In [ ]:
#imputing missing values in outlet_size by mode of outlet size against each outlet type
from scipy.stats import mode
outlet_size_mode= data.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=(lambda x: mode(x).mode[0]))
miss_boolean= data['Outlet_Size'].isnull()
data.loc[miss_boolean,'Outlet_Size']= data.loc[miss_boolean,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
sum(data['Outlet_Size'].isnull())

In [ ]:
outlet_size_mode

In [ ]:
#imputing product visibility with avg visibility of that products since it being zero doesnt make sense
visibility_avg= data.pivot_table(values='Item_Visibility', index='Item_Identifier')
miss_boolean= (data['Item_Visibility']==0)
data.loc[miss_boolean,'Item_Visibility']= data.loc[miss_boolean,'Item_Identifier'].apply(lambda x: visibility_avg[x])
sum(data['Item_Visibility']==0)

In [ ]:
#ratios of visibility as compared to mean visibility as a variable
data['Item_Visibility_MeanRatio']= data.apply(lambda x: x['Item_Visibility']/visibility_avg[x['Item_Identifier']], axis=1)
data['Item_Visibility_MeanRatio'].describe()

In [ ]:
#creating a broad category type of item
data['Item_Type_Combined']= data['Item_Identifier'].apply(lambda x: x[0:2])
data['Item_Type_Combined']= data['Item_Type_Combined'].map({'FD': 'Food', 'NC': 'Non-Consumable', 'DR': 'Drinks'})
data['Item_Type_Combined'].value_counts()

In [ ]:
#determine years of operation of a store
data['Outlet_Years']= 2013- data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

In [ ]:
#modify categories of item fat content and let them be under similarly named categories
data['Item_Fat_Content']= data['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'})
data['Item_Fat_Content'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
data['Outlet']= le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mod:
    data[i]= le.fit_transform(data[i])

In [ ]:
data= pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])

In [ ]:
data.dtypes

In [ ]:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [ ]:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [ ]:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

In [ ]:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [ ]:
#making a function to fit and validate the model
target_train= train.Item_Outlet_Sales
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictor_variables= [x for x in train.columns if x not in [target]+IDcol]
predictors_train= train[predictor_variables]
predictors_test= test[predictor_variables]
#IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn import cross_validation, metrics
#def modelfit(alg, train, test, predictors, target, IDcol):
    #alg.fit(train[predictors],train[target])
    #rain_predictions= alg.predict(train[predictors])
    #cv_score= cross_validation.cross_val_score(alg, train[predictors], train[target], cv=20, scoring= 'mean_squared_error')
    #test[target]= alg.predict(test[predictors])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt= DecisionTreeRegressor(max_depth=8, min_samples_leaf=100)
dt.fit(predictors_train,target_train)
predicted_values= dt.predict(predictors_train)
dt.score(predictors_train,target_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rm= RandomForestRegressor(n_estimators=200, max_depth=7, min_samples_leaf= 100, n_jobs=5)
rm.fit(predictors_train,target_train)
rm.score(predictors_train,target_train)

In [ ]:
predictors_train